In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df.csv')
df_contexts = pd.read_csv('contexts.csv', index_col='id')

In [3]:
df['source'].fillna('testing', inplace=True)
df = df[~df['contexts'].isnull()]
len(df)

45122

In [4]:
df.head()

,participant,source,cs,is_filler,word,group,contexts
0,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,49,True,длинный,no,331 332 333 334 335
1,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,152 154 158
2,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,159 155 151
3,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,24,False,беззаботный,close_far_meton,153 156 157
4,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,43,False,смелый,close_close_meton,297 295


In [5]:
df_contexts.head()

,cs,derivation,text
id,,,
1,1,metonymy,У него были маленькие злые глазки.
9,2,homonym1,Повышение по службе дало надежды на карьерный ...
18,3,literal,В этом году английский у нас преподает нервна...
25,4,homonym1,Наша задача ― посеять на лугах и пастбищах нов...
32,5,literal,"В ходе расследования выяснилось, что водитель ..."


In [6]:
df_fillers = df[df['is_filler']].copy()
df_fillers.head()

,participant,source,cs,is_filler,word,group,contexts
0,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,49,True,длинный,no,331 332 333 334 335
10,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,48,True,круглый,no,325 326 327 328 329 330
19,e462d60e-28f2-4b3f-af8c-ee80342034c4,testing,44,True,строевой,no,300 301 302 303 304 305 306
24,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,48,True,круглый,no,325 326 327 328 329 330
25,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,44,True,строевой,no,300 301 302 303 304 305 306


In [7]:
df_fillers[
    (df_fillers['participant'] == '1a8f0842-6a05-4a24-9235-437b6c35ef73')
    & (df_fillers['cs'] == 46)
]

,participant,source,cs,is_filler,word,group,contexts
27,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,46,True,половой,no,312 313 314
28,1a8f0842-6a05-4a24-9235-437b6c35ef73,testing,46,True,половой,no,315 316 317 318 319


In [8]:
df_fillers['homogeneous_group'] = df_fillers['contexts'].apply(
    lambda cs: len({df_contexts.loc[int(c_id)]['derivation'] for c_id in cs.split()}) == 1)

In [9]:
df_fillers['homogeneous_group'].mean()

0.9608126410835215

In [10]:
all_homogeneous = df_fillers.groupby(['participant', 'source'])['homogeneous_group'].min().reset_index()

In [11]:
all_homogeneous.groupby(['source'])['homogeneous_group'].agg(['count', 'mean'])

,count,mean
source,,
all2,1,1.000000
social,192,0.989583
testing,30,0.566667
toloka,1455,0.858419
toloka_selected,468,0.931624
